In [7]:
# Load the gams extension
%load_ext gams_magic

The gams_magic extension is already loaded. To reload it, use:
  %reload_ext gams_magic


# Piecewise Linear Networks

Consider a transportation network with 3 commodities.
Demand exists between certain origin-destination (OD) pairs for each commodity.

The cost of sending commodities over each arc can be modeled using log(x+1)
where x represents the sum of the flows of each commodity over that arc.
Total flow on each arc must be between 0 and a given capacity.

### By approximating the concave function log(x+1) using a piecewise linear function, formulate and solve this problem for the data given below.

Note that the breakpoints for each log function should be 0,5,10,100.
Use appropriate solver options if necessary 
to ensure the optimal solution is found to within a
2\% relative tolerance, and give lower and upper bounds on the optimal solution
value.  Hand in the log file for this problem.

In [8]:
%%gams
option seed=0; set nodes /1*500/;
parameter offset(nodes); offset(nodes) = round(uniform(2,5));

alias (i,j,nodes); set arcs(nodes,nodes);
arcs(i,j) = no; arcs(i,i+1) = yes; arcs(i,i+offset(i)) = yes;

set k /1*3/;
parameter demand(nodes,k) /
  1.1 -70, 6.1 70, 3.2 -25, 500.2 25, 4.3 -20, 8.3 20, 54.1 -70, 55.1 70
  23.2 -25, 89.2 25, 10.3 -20, 89.3 20, 20.3 -10, 450.3 10 /;

parameter capacity(i,j); capacity(i,j) = uniform(75,85);

In [9]:
%%gams

option optca = 0;
option optcr = 0.4;

set b 'breakpoints' /1*4/;
parameter a(b) /1 0, 2 5, 3 10, 4 100/;

parameter f(b);
f(b) = log(a(b)+1);

display f;

* cantidad de material x transportado por el arc
positive variable x(i,j,k);

* la suma total de todo lo transportado en el arco
positive variable flow(i,j);

SOS2 variable lambda(i,j,b);
variable cost(i,j);


* TotalCost
variable TotalCost;

equation defsoss2; 
defsoss2(arcs).. sum(b,lambda(arcs,b)) =e= 1 ;

equation eqflow;
eqflow(arcs).. flow(arcs) =e= sum(k, x(arcs,k) );

equation eqconvcomb;
eqconvcomb(arcs).. sum(b, lambda(arcs,b)*a(b) ) =e= flow(arcs);

equation eqcost;
eqcost(arcs).. cost(arcs) =e= sum(b, lambda(arcs,b)*f(b));

equation defobj;
defobj.. TotalCost =e= sum(arcs, sum(b, lambda(arcs,b)*f(b)) );

equation eqcapacity;
eqcapacity(arcs).. flow(arcs) =l= capacity(arcs);

equation fbalance;
fbalance(nodes).. sum(i$arcs(i,nodes), flow(i,nodes)) - sum(k, demand(nodes,k) ) =e= sum(j$arcs(nodes,j), flow(nodes,j)) ;

equation xbalance;
xbalance(nodes,k).. sum(i$arcs(i,nodes), x(i,nodes,k)) - demand(nodes,k) =e= sum(j$arcs(nodes,j), x(nodes,j,k)) ;


model traffic /all/;
solve traffic min TotalCost using MIP;
  
display flow.l, x.l, cost.l, TotalCost.l; 


,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Integer (8),499.9005,6971,8947,MIP,CPLEX,111.157


In [17]:
# print the cost and the (total) flow on all arcs here
%gams_pull -d TotalCost flow cost demand
display(TotalCost)
display(flow[flow.level > 0])
display(cost[cost.level > 0])

,level,marginal,lower,upper,scale
0,499.900503,0.0,-inf,inf,1.0


,nodes,nodes,level,marginal,lower,upper,scale
1,1,4,70.0,0.0,0.0,inf,1.0
5,3,7,25.0,0.0,0.0,inf,1.0
6,4,5,70.0,0.0,0.0,inf,1.0
7,4,7,20.0,0.0,0.0,inf,1.0
8,5,6,70.0,0.0,0.0,inf,1.0
...,...,...,...,...,...,...,...
977,489,494,25.0,0.0,0.0,inf,1.0
987,494,497,25.0,0.0,0.0,inf,1.0
991,497,498,25.0,0.0,0.0,inf,1.0
992,498,499,25.0,0.0,0.0,inf,1.0


,nodes,nodes,level,marginal,lower,upper,scale
1,1,4,3.876045,0.0,-inf,inf,1.0
5,3,7,2.767433,0.0,-inf,inf,1.0
6,4,5,3.876045,0.0,-inf,inf,1.0
7,4,7,2.644254,0.0,-inf,inf,1.0
8,5,6,3.876045,0.0,-inf,inf,1.0
...,...,...,...,...,...,...,...
977,489,494,2.767433,0.0,-inf,inf,1.0
987,494,497,2.767433,0.0,-inf,inf,1.0
991,497,498,2.767433,0.0,-inf,inf,1.0
992,498,499,2.767433,0.0,-inf,inf,1.0
